In [0]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

drive.mount('/content/gdrive', force_remount = True)
dataset_path = 'gdrive/My Drive/Deep Learning/MNIST with CNN/'

Using TensorFlow backend.


Mounted at /content/gdrive


In [0]:
train_path = dataset_path + "train.csv"
test_path = dataset_path + "test.csv"

In [0]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [0]:
Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

X_train = X_train / 255.0
test = test / 255.0

X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [0]:
Y_train = to_categorical(Y_train, num_classes = 10)

In [0]:
np.random.seed(2)
random_seed = 2
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)

In [0]:
model = Sequential()

model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [0]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0,
                 amsgrad=False)

In [0]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [0]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [0]:
epochs = 30
batch_size = 128

In [0]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [0]:
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

Epoch 1/30
 - 9s - loss: 0.4763 - acc: 0.8457 - val_loss: 0.0715 - val_acc: 0.9786
Epoch 2/30
 - 8s - loss: 0.1435 - acc: 0.9572 - val_loss: 0.0528 - val_acc: 0.9831
Epoch 3/30
 - 8s - loss: 0.1132 - acc: 0.9661 - val_loss: 0.0485 - val_acc: 0.9845
Epoch 4/30
 - 9s - loss: 0.0887 - acc: 0.9731 - val_loss: 0.0311 - val_acc: 0.9907
Epoch 5/30
 - 8s - loss: 0.0748 - acc: 0.9780 - val_loss: 0.0409 - val_acc: 0.9881
Epoch 6/30
 - 8s - loss: 0.0697 - acc: 0.9785 - val_loss: 0.0253 - val_acc: 0.9924
Epoch 7/30
 - 8s - loss: 0.0650 - acc: 0.9803 - val_loss: 0.0319 - val_acc: 0.9907
Epoch 8/30
 - 8s - loss: 0.0576 - acc: 0.9833 - val_loss: 0.0302 - val_acc: 0.9921
Epoch 9/30
 - 8s - loss: 0.0566 - acc: 0.9826 - val_loss: 0.0307 - val_acc: 0.9902

Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 10/30
 - 8s - loss: 0.0452 - acc: 0.9863 - val_loss: 0.0182 - val_acc: 0.9936
Epoch 11/30
 - 8s - loss: 0.0410 - acc: 0.9872 - val_loss: 0.0168 - val_acc: 0.9957
Epoc

In [0]:
results = model.predict(test)#predict for the test data

# take the index of highest probability 
results = np.argmax(results,axis = 1)
#Convert the array to a pandas series
results = pd.Series(results,name="Label")

In [0]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)